**mandatoryPropertyMissing.ipynb**

EDAM concept is missing a mandatory annotation propery

**Documentation:** https://github.com/edamontology/edamverify/blob/master/docs/mandatoryPropertyMissing.md

Set constants for script return values. Load EDAM_dev.owl from GitHub into an RDF graph.

In [29]:
import sys
from rdflib import ConjunctiveGraph, Namespace

# Constants for script return value as per https://github.com/edamontology/edamverify.
NOERR = 0
INFO  = 1
WARN  = 2
ERROR = 3

#Load EDAM_dev.owl from GitHub into an RDF graph.
print("Loading graph ...", end="")
g = ConjunctiveGraph()
g.load('https://raw.githubusercontent.com/edamontology/edamontology/master/EDAM_dev.owl', format='xml')
# g.load('EDAM_dev.owl', format='xml')
g.bind('edam', Namespace('http://edamontology.org#'))
print("done!")



Loading graph ...done!


Define SPARQL query to retrieve ID, term, and all other properties that are mandatory for Operation, Data, Identifier, Format or Topic concepts. Run the query.

**NB.1:** BASE is used to define the define the default namespace (for various elements below).

**NB.2:** Concepts are identified by one of ``operation``, ``data``, ``format`` or ``topic`` occurring in the concept ID.

In [30]:
# Compile SPARQL query
query_term = """
BASE <http://edamontology.org/>
SELECT ?id ?term ?subsetpath ?definition ?created_in WHERE
{
?id rdfs:label ?term .
?id oboInOwl:inSubset ?subsetpath .
OPTIONAL {?id oboInOwl:hasDefinition ?definition .}
OPTIONAL {?id :created_in ?created_in .}
}
"""

# Declare hash tables for results
# ids is for concept IDs
# edamSubset is used to record 'edam' subset assignations
# branchSubset is used to record 'topics', 'data', 'formats' or 'operations' subset assignations
edamSubset = {}
branchSubset = {}
ids = {}
terms = {}

# Run SPARQL query and collate results
err = NOERR
errfound = False
report = list()
results = g.query(query_term)

Analyse results of query.

In [31]:
# Error message building function
def report_append(err_new, msg):
    """
    Writes a report of an error and sets the error level appropriately.
    err: the error level to set
    msg: the message to write
    """
    global err
    global errfound
    global report
    if err_new > err:
        err = err_new
    errfound = True
    report.append(msg)

for r in results :
#    print(str(r['id']), str(r['term']), str(r['ext']), str(r['exact_syn']))
    id   = str(r['id'])
    term = str(r['term'])
    subsetpath  = str(r['subsetpath'])
    definition = str(r['definition'])
    created_in = str(r['created_in'])


    # Skip obsolete concepts
    if subsetpath != 'None' and "obsolete" in subsetpath:
        continue

    # Check for Format concepts
    if "operation_" in id \
        or "data_" in id \
        or ("format_" in id and "is_format_of" not in id)\
        or ("topic_" in id and "is_topic_of" not in id):

        # Update the IDs hash table
        ids[id] = id
        terms[id] = term

        # Update the subset assignation hash tables
        if "edam" in subsetpath:
            edamSubset[id] = "true"
        if "topics" in subsetpath \
            or "operations" in subsetpath \
            or "data" in subsetpath \
            or "identifiers" in subsetpath \
            or "formats" in subsetpath:
            branchSubset[id] = "true"

        # Check for missing 'definition' and 'created_in' properties
        if definition == "None":
            report_append(ERROR, "ERROR Missing property (definition) on ::: " + id +  ' (' + term + ')')
        if created_in == "None":
            report_append(ERROR, "ERROR Missing property (created_in) on ::: " + id +  ' (' + term + ')')

# Check for missing subset properties
for next_id in ids:
    if next_id not in edamSubset:
        report_append(ERROR, "ERROR 'edam' subset assignation missing on ::: " + next_id +  ' (' + terms[next_id] + ')')
    if next_id not in branchSubset:
        report_append(ERROR, "ERROR 'topics', 'operations', 'data' or 'formats' subset assignation missing on ::: " + next_id +  ' (' + terms[next_id] + ')')

Write report and return approriate value.

In [32]:
# Return exit code (raises exception)
if errfound:
    print('Mandatory properties missing from these concepts:')
    print("\n".join(report))
    sys.exit(err)
else:
    print("No issues found.")
    sys.exit(NOERR)

Mandatory properties missing from these concepts:
ERROR 'topics', 'operations', 'data' or 'formats' subset assignation missing on ::: http://edamontology.org/format_3623 (Index format)


SystemExit: 3

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
